In [137]:
import pymysql

In [144]:
#grab the entire table from sql
#contains recent traded option log and updated gex/vex/ddoi values
#make this take an argument in case we have more tables in the future
def get_sql_df():
    Host = 'deribit-db-instance.csduabs5w5go.us-east-1.rds.amazonaws.com'
    User = 'admin'
    Password = 'Deribit123'
    Database = 'deribit'
    con = pymysql.connect(user=User, password=Password,host=Host, database=Database, port=3306, ssl_key='../../Downloads/deribit-key-pair.pem')
    try:
        with con.cursor() as cur:
            query = 'SELECT * FROM deribit.option_table'
            cur.execute(query)
            df = pd.read_sql(query, con)
            return df
    finally:
        con.close()

In [145]:
df = get_sql_df()
df.head()

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,buy,10,10,10,0
1,0,2,64649826,1613851501155,0,0.1340,0.073299,165.31,BTC-5MAR21-57000-P,56738.42,buy,3,20,15,0
2,0,50,64649954,1613851546122,0,0.1370,0.063862,198.70,BTC-5MAR21-57000-C,56731.02,buy,3,30,16,0
3,0,57,64650074,1613851603336,0,0.0695,0.061493,134.11,BTC-5MAR21-60000-C,56796.98,buy,3,40,20,0
4,0,336,64657047,1613855054463,2,0.0560,0.151207,39.36,BTC-26FEB21-54000-C,56715.78,sell,2,30,10,0


In [160]:
new_trade = df.iloc[-1]
new_trade
#call get_sql_df to get dataframe of all options
#call get_instrument on each instrument_name to see if its active or not
#create new dataframe based on that
#add new_trade to new dataframe
#call get_order_book on the new dataframe (since it has only active options)
#calculate new gamma/vex/ddoi
#MAYBE: remove any old sql stuff and store them?
#post new trade info + gamma/vex/ddoi

MyUnknownColumn                      0
trade_seq                            2
trade_id                      64657489
timestamp                1613857287961
tick_direction                       2
price                             0.01
mark_price                  0.00300078
iv                              114.51
instrument_name    BTC-21FEB21-56000-P
index_price                    56704.4
direction                          buy
amount                               1
gex                                 10
vex                                 20
ddoi                                 0
Name: 5, dtype: object

In [173]:
import urllib.request, json
#/public/get_instrument
url = 'https://test.deribit.com/api/v2/public/get_instrument?instrument_name='
url += new_trade[8]
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())
result = data['result']
print(result)
if(result['is_active']):
    print("active")
else:
    print("not active")

{'tick_size': 0.0005, 'taker_commission': 0.0003, 'strike': 56000.0, 'settlement_period': 'day', 'quote_currency': 'USD', 'option_type': 'put', 'min_trade_amount': 0.1, 'maker_commission': 0.0003, 'kind': 'option', 'is_active': False, 'instrument_name': 'BTC-21FEB21-56000-P', 'expiration_timestamp': 1613894400000, 'creation_timestamp': 1613721603000, 'contract_size': 1.0, 'block_trade_commission': 0.00015, 'base_currency': 'BTC'}
not active
